# Lab 8: Graphical Models

The goal of this lab session is to code two methods to estimate the structure of undirected gaussian graphical models and compare them.

You have to send the filled notebook named **"L8_familyname1_familyname2.ipynb"** (groups of 2) by email to aml.centralesupelec.2019@gmail.com before December 12, 2019 at 23:59 and put **"AML-L8"** as subject. 

We begin with the standard imports:

In [31]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import sklearn
%matplotlib inline
plot_kwds = {'alpha' : 0.25, 's' : 80, 'linewidths':0}

## Graphical Models

A graphical model is a probabilistic model for which a graph expresses the conditional dependence structure between random variables. The variables are represented by nodes and the relations between them are represented by edges.

### GLasso

Graphical Lasso is the name of the optimization problem that estimates the precision matrix of a multivariate gaussian and its name comes from the direct link with graphical models and the regularization term. 

Fill in the following class that implements the GLasso algorithm optimized by ADMM:

In [101]:
class my_GLasso():
    
    def __init__(self, alpha, mu, max_iter = 60, tol=1e-4):
        '''
        Parameters:
        alpha : float
            Penalization parameter selected.
        mu: float>0

        Attributes:
        
        covariance_ : numpy.ndarray, shape (n_features, n_features)
            Estimated covariance matrix.
        precision_ : numpy.ndarray, shape (n_features, n_features)
            Estimated precision matrix (inverse covariance).
        '''
        self.covariance_ = None
        self.precision_ = None
        self.alpha_ = alpha
        self.mu_ = mu
        self.iter_max_ = max_iter
        self.tol_ = tol
        
        self.history_ = None
        
    def update_precision_(self, S, Z, V, mu):
        """
        Performs an update of the precision matrix P
        
        Arguments:
        ----------
        S: (n_features, n_features) numpy.ndarray
            Empirical covariance matrix
        
        Z: (n_features, n_features) numpy.ndarray
            Dual variable for precision matrix
        
        V: (n_features, n_features) numpy.ndarray
            Duality jump
        
        mu: float > 0
            
        Returns:
        --------
        P: (n_features, n_features) numpy.ndarray
            The precision matrix minimizing the gradient of Lagragian wrt P
        """
        T = Z - V - (1/mu)*S
        eig_values, eig_vectors = np.linalg.eig(T)
        new_eig_values = (eig_values + np.sqrt(eig_values**2 + 4/mu))/2
        return eig_vectors.T @ np.diag(new_eig_values) @ eig_vectors
    
    def update_dual_(self, P, S, V, alpha, mu):
        """
        Performs an update of the precision matrix dual Z
        
        Arguments:
        ----------
        S: (n_features, n_features) numpy.ndarray
            Empirical covariance matrix
        
        P: (n_features, n_features) numpy.ndarray
            Precision matrix estimate
        
        V: (n_features, n_features) numpy.ndarray
            Duality jump
        
        alpha: float > 0
            Regularization parameter
        
        mu: float > 0
            
        Returns:
        --------
        Z: (n_features, n_features) numpy.ndarray
            The dual precision matrix minimizing the gradient of Lagragian wrt Z
        """
        return np.multiply(np.sign(P+V), np.maximum(0, np.absolute(P + V) - alpha/mu))
    
    def update_jump_(self, old_V, P, Z, mu):
        """
        Updates the duality jump V
        
        Arguments:
        ----------
        old_V: (n_features, n_features) numpy.ndarray
            Old duality jump
        
        P: (n_features, n_features) numpy.ndarray
            Precision matrix estimate
            
        Z: (n_features, n_features) numpy.ndarray
            Dual variable for precision matrix
        
        mu: float > 0
        
        Returns:
        --------
        (n_features, n_features) numpy.ndarray
            New value for the duality jump
        """
        return old_V + mu*(P-Z)
        
    def fit(self, X):
        """ Fits the GraphicalLasso model to X.
        
        Parameters:
        -----------
        X: (n, p) np.array
            Data matrix
        
        Returns:
        -----
        self
        """
        n_samples, n_features = X.shape
        
        # Compute empirical estimators for esperance and covariance
        esperance = np.mean(X, axis=0)
        
        # TODO - optimize this calculation
        S = 0
        for i in range(n_samples):
            dev = (X[i] - esperance).reshape(-1,1)
            S += dev@dev.T
        S /= n_samples
        
        # Initialize latent variable to definite positive symetric matrices
        Zn = np.eye(n_features)
        Vn = np.eye(n_features)
        Pn = np.eye(n_features)
        
        self.history_ = {
            "precision": [Pn],
            "dual": [Zn],
            "jump": [Vn],
        }
                
        converged = False
        itr = 0
        
        while itr < self.iter_max_ and not converged:
            
            Pnp1 = self.update_precision_(S, Zn, Vn, self.mu_)
            Znp1 = self.update_dual_(Pnp1, S, Vn, self.alpha_, self.mu_)
            Vnp1 = self.update_jump_(Vn, Pnp1, Znp1, self.mu_)
            
            converged = np.linalg.norm(Pn - Pnp1) < self.tol_ * np.linalg.norm(Pn)
        
            Pn = Pnp1
            Vn = Vnp1
            Zn = Znp1
            
            self.history_["precision"].append(Pn)
            self.history_["dual"].append(Zn)
            self.history_["jump"].append(Vn)
            itr += 1
        
        self.precision_ = Zn
        self.covariance_ = np.linalg.inv(Zn)
        return self        

### Nodewise Regression

Fill in the following class that implements the nodewise regression algorithm to estimate a graphical model structure. You can use `LassoCV` for the regressions. Bonus (not graded): Implement your own cross-validation lasso.

In [42]:
from sklearn.linear_model import LassoCV

class my_nodewise_regression():
    
    def __init__(self, rule, alpha):
        '''
        Parameters:
        
        rule: {"OR", "AND"}
        alpha: float
            regularization parameter
        
        Attributes:
        
        covariance_structure_ : numpy.ndarray, shape (n_features, n_features)
            Estimated covariance matrix.        
        '''
        self.graph_structure_ = None
        self.rule_ = rule
        self.alpha_ = alpha
        
    def fit(self, X):
        """ Fit the model to X.
        
        Parameters:
        -----------
        X: (n, p) np.array
            Data matrix
        
        Returns:
        -----
        self
        """
        n_features = X.shape[1]
        beta = np.zeros((n_features, n_features))
        for j in range(n_features):
            y = X[j]#.reshape(-1,1)
            x = np.delete(X, j, axis=0)
            regressor = sklearn.linear_model.Lasso(alpha=self.alpha_)
            regressor.fit(X=x, y=y)
            beta[j, :j], beta[j, j+1:] = regressor.coef_[:j], regressor.coef_[j:]
        
        domain_estimate = beta > 1e-7
        
        if self.rule_ == "OR":
            adj = beta | beta.T
        
        elif self.rule == "AND":
            adj = beta & beta.T
        
        self.graph_structure_ = adj
        return self

Generate an easy-to-check (non-trivial, p<=6) example and plot the 4 (real, GLasso, AND, OR) graphs. You can use `networkx` to plot the resulting graph.

In [85]:
mu=[0., 0., 0.]
cov = np.array([
    [1., 0., 1.],
    [0., 1., 0.],
    [1., 0., 1.]
])
cov = cov.T@cov

X = np.array([np.random.multivariate_normal(mu, cov) for _ in range(100)])


In [102]:
model = my_GLasso(alpha=0.1, mu=1)

model.fit(X)

G = nx.from_numpy_array(model.covariance_)

In [105]:
model.covariance_

array([[ 2.24612866, -0.79599848, -0.0751022 ],
       [-0.79599848,  1.6979309 , -0.02150432],
       [-0.0751022 , -0.02150432,  0.58288455]])

## Simulations

Compare the two graph estimators for the following model with $p = 300$ and $n = 40, 80, 320$:

- An AR(1)-Block model. In this model the *covariance* matrix is block-diagonal with equalsized AR(1)-blocks of the form $(\Sigma_{Block})_{i, j} = 0.9^{|i−j|}$, take $30 \times 30$ blocks.

Report accuracy and F1 score for the edge estimation.

For GLasso estimation, use cross-validation k-fold with loglikelihood loss to select the $\lambda$ penalization parameter.

Measure the estimation error of the GLasso matrix result.

In [ ]:
# TODO